In [1]:
#Using pyftpdlib to create server
import socket
import os
import time
from PIL import Image
import numpy as np
from pyftpdlib.authorizers import DummyAuthorizer
from pyftpdlib.handlers import FTPHandler
from pyftpdlib.servers import FTPServer

In [2]:
def getIP():
    myname = socket.getfqdn(socket.gethostname())
    get_s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    get_s.connect(('8.8.8.8', 0))
    ip = get_s.getsockname()[0]
    return ip

In [3]:
IP = getIP()

In [ ]:
from keras.models import load_model
model =load_model('model_01.h5')

Using TensorFlow backend.


In [ ]:
def EvaluateImage(file):
    #100*100
    ima = Image.open(file);
    ima = ima.resize((100, 100))
    
    r2 = 100 
    imb = Image.new('RGB', (r2, r2),(0))
    pima = ima.load()
    pimb = imb.load()
    r = float(r2/2) 
    for i in range(r2):
        for j in range(r2):
            lx = abs(i-r+0.5) 
            ly = abs(j-r+0.5) 
            l  = pow(lx,2) + pow(ly,2)
            if l <= pow(r, 2):
                    pimb[i,j] = pima[i,j]
                    
    arr = np.asarray(imb, dtype = np.float32)
    arr_flo = arr/255
    x_test = arr_flo.reshape(1,100,100,3)
    
    probility = model.predict(x_test)
    probility=probility.reshape(18)
    
    max_label=0
    for j in range(1,18):
        if probility[j] > probility[max_label]:
            max_label = j
    
    result = str(max_label)
    f=open(file + ".txt", 'w')
    for i in range(0,len(result)):
        if result[i] >= '0' and result[i] <= '9':
            f.write(result[i])
        else:
            break
    f.close()
    del ima, imb, x_test, probility, result

In [ ]:
class MyHandler(FTPHandler):

    def on_connect(self):
        print ("%s:%s connected" % (self.remote_ip, self.remote_port))
        pass

    def on_disconnect(self):
        # do something when client disconnects
        pass

    def on_login(self, username):
        # do something when user login
        pass

    def on_logout(self, username):
        # do something when user logs out
        pass

    def on_file_sent(self, file):
        # do something when a file has been sent
        pass

    def on_file_received(self, file):
        # do something when a file has been received
        EvaluateImage(file)
        pass

    def on_incomplete_file_sent(self, file):
        # do something when a file is partially sent
        pass

    def on_incomplete_file_received(self, file):
        # remove partially uploaded files
        import os
        os.remove(file)

In [ ]:
def main():
    # Instantiate a dummy authorizer for managing 'virtual' users
    authorizer = DummyAuthorizer()

    # Define a new user having full r/w permissions and a read-only
    # anonymous user
    authorizer.add_user('userf', 'f12345', '/FTP/flower', perm='elradfmwMT')
    authorizer.add_anonymous('/FTP/anonymous')

    # Instantiate FTP handler class
    handler = MyHandler
    handler.authorizer = authorizer

    # Define a customized banner (string returned when client connects)
    #handler.banner = "pyftpdlib based ftpd ready."

    # Specify a masquerade address and the range of ports to use for
    # passive connections.  Decomment in case you're behind a NAT.
    #handler.masquerade_address = IP
    handler.passive_ports = range(60000, 65535)

    # Instantiate FTP server class and listen on 127.0.0.1:21
    address = (IP, 21)
    server = FTPServer(address, handler)

    # set a limit for connections
    server.max_cons = 256
    server.max_cons_per_ip = 5
    
    # start ftp server
    server.serve_forever()

In [ ]:
if __name__ == '__main__':
    main()